<a href="https://colab.research.google.com/github/Akhmadkhan/Research-4/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Эксперимент с параметрами РНН

Перевод символов в двоичный вид

In [20]:
import random
def char_to_binary(char):
    return format(ord(char), '08b')

# Символы A и B в двоичном виде
A = char_to_binary('A')  # '01000001'
B = char_to_binary('B')  # '01000010'
random.seed(42)

# Создание списка из 1000 перемешанных в разном порядке двоичных представлений A и B
sequence = [A, B] * 500
random.shuffle(sequence)

# Сохранение последовательности в файл
with open('binary_sequence.txt', 'w') as file:
    for binary_char in sequence:
        file.write(binary_char + '\n')

In [28]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Загрузка данных из файла
with open('binary_sequence.txt', 'r') as file:
    data = file.readlines()

# Преобразование данных в формат для обучения модели
X = np.array([list(map(int, line.strip())) for line in data])
y = np.array([1 if line.strip() == '01000001' else 0 for line in data])

# Создание и обучение модели LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(X.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.expand_dims(X, axis=2), y, epochs=10, batch_size=32)

# Тестирование модели на входе A = 01000001
test_input = np.array([[0, 1, 0, 0, 0, 0, 0, 1]])  # Вход A
predicted_output = model.predict(np.expand_dims(test_input, axis=2))
print("Предсказание для A: {:.2f}%".format(predicted_output[0][0] * 100))

Epoch 1/10
32/32 [==============================] - 3s 6ms/step - loss: 0.6749 - accuracy: 0.7980
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 0.6041 - accuracy: 1.0000
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.2921 - accuracy: 1.0000
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0208 - accuracy: 1.0000
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 6/10
32/32 [==============================] - 0s 6ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 7/10
32/32 [==============================] - 0s 7ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 8/10
32/32 [==============================] - 0s 8ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 9/10
32/32 [==============================] - 0s 8ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 470ms/step
Предсказание для A: 99.89%


In [41]:
from keras.datasets import mnist
from keras.utils import to_categorical

# Загрузка данных
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Предобработка данных: преобразование в двоичный вид
X_train_binary = (X_train > 127).astype(np.int_)
X_test_binary = (X_test > 127).astype(np.int_)

# Преобразование формы данных для LSTM
time_steps = X_train_binary.shape[1]
num_features = X_train_binary.shape[2]
X_train_lstm = X_train_binary.reshape(-1, time_steps, num_features)
X_test_lstm = X_test_binary.reshape(-1, time_steps, num_features)

# Создание LSTM модели
model = Sequential([
    LSTM(128, input_shape=(time_steps, num_features)),
    Dense(10, activation='softmax')
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(X_train_lstm, y_train, epochs=5, batch_size=32, validation_split=0.2)

# Оценка модели на обучающем и контрольном датасете
train_loss, train_acc = model.evaluate(X_train_lstm, y_train)
test_loss, test_acc = model.evaluate(X_test_lstm, y_test)

print(f'Accuracy on training data: {train_acc}')
print(f'Accuracy on test data: {test_acc}')

# Проверка на одной случайной картинке из обучающего датасета
index = np.random.randint(0, len(X_train_lstm))
sample_image = X_train_lstm[index]
sample_label = y_train[index]

prediction = model.predict(np.expand_dims(sample_image, axis=0))
predicted_label = np.argmax(prediction)

print(f'Actual label: {sample_label}, Predicted label: {predicted_label}')

# Проверка на одной случайной картинке из контрольного датасета
index = np.random.randint(0, len(X_test_lstm))
sample_image = X_test_lstm[index]
sample_label = y_test[index]

prediction = model.predict(np.expand_dims(sample_image, axis=0))
predicted_label = np.argmax(prediction)

print(f'Actual label: {sample_label}, Predicted label: {predicted_label}')

Epoch 1/5
1500/1500 [==============================] - 66s 42ms/step - loss: 0.3922 - accuracy: 0.8723 - val_loss: 0.1442 - val_accuracy: 0.9570
Epoch 2/5
1500/1500 [==============================] - 64s 43ms/step - loss: 0.1262 - accuracy: 0.9617 - val_loss: 0.1060 - val_accuracy: 0.9678
Epoch 3/5
1500/1500 [==============================] - 67s 45ms/step - loss: 0.0865 - accuracy: 0.9740 - val_loss: 0.0953 - val_accuracy: 0.9718
Epoch 4/5
1500/1500 [==============================] - 69s 46ms/step - loss: 0.0684 - accuracy: 0.9787 - val_loss: 0.0742 - val_accuracy: 0.9781
Epoch 5/5
313/313 [==============================] - 7s 21ms/step - loss: 0.0634 - accuracy: 0.9806
Accuracy on training data: 0.9850166440010071
Accuracy on test data: 0.9805999994277954
1/1 [==============================] - 0s 488ms/step
Actual label: 2, Predicted label: 2
1/1 [==============================] - 0s 30ms/step
Actual label: 8, Predicted label: 8
